[View in Colaboratory](https://colab.research.google.com/github/tcglarry/alibaba_manufacturing_competition/blob/master/ali_xgb_densenet121.ipynb)

In [2]:
"""For Alibaba Competition

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/10DYDNSM_eaFtn7lrM0CyUIXRFdIDtabS
"""

# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse






Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp 'drive/My Drive/kaggle/ali/new_era/new.zip' .

In [5]:

!unzip new.zip


Archive:  new.zip
   creating: new/
  inflating: new/横条_aug_凹20180903110605对照样本_02_s_0_9518.png  
  inflating: new/凸粉_aug_0180901101356对照样本_02_s_0_1493.png  
  inflating: new/擦花_aug_0180901135948对照样本_01_s_0_3879.png  
  inflating: new/漏底_aug_0180901094548对照样本_02_s_0_2505.png  
  inflating: new/脏点20180910093625对照样本_02_small.png  
  inflating: new/擦花_aug_0180915110340对照样本_02_s_0_8072.png  
  inflating: new/碰伤_aug_0180905093918对照样本_02_s_0_766.png  
  inflating: new/起坑_aug_0180905110015对照样本_02_s_0_1504.png  
  inflating: new/其他_aug_拖烂20180910101305对照样本_02_s_0_1797.png  
  inflating: new/凸粉_aug_0180901101631对照样本_02_s_0_3131.png  
  inflating: new/擦花_aug_0180901105534对照样本_02_s_0_1674.png  
  inflating: new/脏点_aug_0180910092754对照样本_02_s_0_2061.png  
  inflating: new/其他_aug_碰凹20180915102751对照样本_02_s_0_2319.png  
  inflating: new/擦花_aug_0180901141725对照样本_02_s_0_686.png  
  inflating: new/其他_aug_碰凹20180912101411对照样本_02_s_0_8028.png  
  inflating: new/脏点_aug_0180911135453对照样本_02_s_0_4323.png  
  

In [0]:
!rm -rf new.zip

In [1]:
!ls

sample_data


In [7]:
!pip install --upgrade keras
!pip install --upgrade tensorflow

    100% |████████████████████████████████| 317kB 7.0MB/s 
  Found existing installation: Keras 2.1.6
    Uninstalling Keras-2.1.6:
      Successfully uninstalled Keras-2.1.6
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (1.11.0)


In [1]:

import keras
print (keras.__version__)
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import subprocess
import os
import pickle
import sklearn 

from sklearn.model_selection import train_test_split

from xgboost.sklearn import XGBClassifier




import keras.backend as K

from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
#from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenetv2 import MobileNetV2,preprocess_input
#from keras.applications.nasnet import NASNetMobile,NASNetLarge
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.preprocessing.image import ImageDataGenerator,  img_to_array, load_img



#from keras.applications.xception import Xception, preprocess_input

from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, Reshape, Dropout,Cropping2D,Activation
from keras.models import Model, load_model
import pandas as pd

from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.callbacks import Callback


from keras import regularizers
#from dummyPy import OneHotEncoder
from keras.utils import to_categorical

from sklearn.metrics import confusion_matrix

import itertools
import time

Using TensorFlow backend.


2.2.0


In [10]:
src_1= 'manu_big_data/'
src_2= 'manu_big_data/new_era/data_aug_gen/new/'
src_save = 'manu_big_data/new_era/'

In [4]:
train_file_list = os.listdir(src_2)
if '.DS_Store' in train_file_list:
    train_file_list.remove('.DS_Store')
    print ('.DS_Store deleted')
    
if '.DS_Store' in train_file_list:
    print ('wrong')
else:
    print ('ok')
    

ok


In [5]:
print (len(train_file_list))


17411


In [6]:
with open (src_save+'train_file_list_aug.pkl','wb') as handle:
  pickle.dump(train_file_list, handle)
print ('train file list saved')

train file list saved


In [4]:
with open (src_save+'train_file_list_aug.pkl','rb') as handle:
  train_file_list = pickle.load(handle)
print ('train file list loaded')

train file list loaded


In [11]:

with open(src_1+'failure_to_label.pkl', 'rb') as handle:
    label_dict = pickle.load(handle)
label_dict  


{'正常': 0,
 '不导': 1,
 '擦花': 2,
 '横条': 3,
 '桔皮': 4,
 '漏底': 5,
 '碰伤': 6,
 '起坑': 7,
 '凸粉': 8,
 '涂层': 9,
 '脏点': 10,
 '其他': 11}

In [12]:
# label the image according to the defect type
label_data = []
for f in train_file_list:
    label_data.append(label_dict[f[:2]])
train_label_data = np.array(label_data)
label_df = pd.DataFrame (train_label_data)
label_df.columns = ['label']
print (len(label_df))
print (label_df['label'].value_counts())

17411
2     3626
11    3517
5     2716
6     1693
10    1463
3     1034
1      858
9      768
8      728
4      630
7      378
Name: label, dtype: int64


In [13]:
with open (src_save+'train_label_data_aug.pkl','wb') as handle:
  pickle.dump(train_label_data, handle)
print ('train label data saved')

train label data saved


In [6]:
with open (src_save+'train_label_data_aug.pkl','rb') as handle:
  train_label_data = pickle.load(handle)
print ('train label data loaded')

train label data loaded


In [0]:
#train_x,val_x,train_y,val_y = train_test_split(train_data,trai_label_data_0,test_size=0.2,shuffle=True)

In [0]:
#print (train_x.shape)

In [14]:
def build_model():
    #inputs = Input(shape=(1920,2560,3))
    inputs = Input(shape=(240,320,3))
   
    # create the base pre-trained model
    #base_model = DenseNet121(weights='imagenet',input_tensor=inputs,  include_top=True)
    base_model = MobileNetV2(weights='imagenet',input_tensor=inputs,  include_top=True)
    #base_model =load_model('drive/My Drive/'+'MobileNetV2.h5')
  
  

 
    ''' 
    model_1 = Model(inputs=inputs, outputs=base_model.output)
    model_1.summary()
    '''
    
    
    print ('\n\n*******************************=============================')
    model_2_out = base_model.layers[-3].output
    
    model_2 = Model(inputs=inputs, outputs=model_2_out)
    
    model_2.summary()
    
    return model_2





In [15]:
#model_1, model_2 = build_model()

model_2= build_model()
model_2.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
print ('model compile complete')



*******************************=============================
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 240, 320, 3)  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 120, 160, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 120, 160, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (Activation)         (None, 120, 160, 32) 0           bn_Conv1[0][0]                   
______________________________________________

for i, layer in enumerate(model_1.layers):
    print (f'layer ({i}) layer name is {layer.name}')


checkpoint = ModelCheckpoint(filepath=src_1+'defect_non_defect_dense.h5', monitor = 'val_loss', save_best_only=True, mode= 'auto')
earlystop = EarlyStopping(patience=10)
callback_list = [checkpoint, earlystop]
model_1.fit(x=train_x, y=train_y, batch_size=32, epochs=300, callbacks = callback_list, verbose=1, validation_split=0.2,  shuffle=True)


In [0]:
model_1.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
def first_prediction(model,data):
    pred_temp = model.predict(data)
    return pred_temp

In [17]:
# assign batch inferefcing of pre-traianed model to be 4 

def make_batch (file_list,batch_size):
  final_list =[]
  
 
  floor = int(np.floor(len(file_list)/batch_size))
  print ('floor=', floor)
  for i in range(floor+1):
      if i < floor:
          final_list.append(file_list[i*batch_size:(i+1)*batch_size])
      else: 
          final_list.append(file_list[i*batch_size:])
          
  # check final_llist len equal to file_list  
  a =0 
  for l in final_list:
    #print (len(l))
    #print (l[-10:])
    a += len(l)
  if (a - len(final_list) == 0):  
    print ('total size', a, 'final_list length equal to file_list')
  else:
    print ('total size', a, 'file_list', len(file_list),'final_list length NOT equal to file_list')
    print (a - len(file_list))
    print ('warning, something wrong')
    
  return final_list  



In [18]:
batch_size = 32
final_list = make_batch(train_file_list,batch_size)
print (len(final_list))

floor= 544
total size 17411 file_list 17411 final_list length NOT equal to file_list
0
warning, something wrong
545


In [44]:
s= time.time()
final_array = []
for j,l in enumerate(final_list):
      img_list=[]
      for i,img in enumerate(l):
        
        
        
        img = load_img('new/'+img)
        
        if i == 0:
            print (img.size)
        '''    
        img = img.resize((2560//4,1920//4))
        if i == 0:
            print (img.size)
            
        '''
        
        x = img_to_array(img)
        
        
        
        x = x/255.
        

        if i  == 0 :
          
          print (x.shape)
          #x = cv2.cvtColor(y, cv2.COLOR_BGR2RGB)
          #x = x/127.5 -1.
        img_list.append(x)
      #print ('img_list=', len(img_list))
      testing_data = np.array(img_list)

      #print ('testing_data_shape', testing_data.shape)
      #train_data =  train_data[:,:,:,np.newaxis]
      #print (testing_data.shape)
      pred_temp = first_prediction(model_1,testing_data)

      print('batch = ', j, 'stage 1 completed', 'shape=', pred_temp.shape)
      #np.save(src_save+'temp_'+str(j)+'.npy',pred_temp)

      #print ('bacth npy saved', j)
      e = time.time()
      print ('batch',j,'time=', round(e -s ,4))

      final_array.append(pred_temp)
      print ('afterbatch',j,'length=', len(final_array))

final_array = np.concatenate(final_array)
print ('final array', final_array.shape)
np.save(src_save+'train_data_MobileNetV2_1_aug.npy',final_array)
print ('totally done, train_data_MobileNetV2_1_aug.npy')

(320, 240)
(240, 320, 3)
batch =  0 stage 1 completed shape= (32, 1000)
batch 0 time= 3.3258
afterbatch 0 length= 1
(320, 240)
(240, 320, 3)
batch =  1 stage 1 completed shape= (32, 1000)
batch 1 time= 3.6309
afterbatch 1 length= 2
(320, 240)
(240, 320, 3)
batch =  2 stage 1 completed shape= (32, 1000)
batch 2 time= 3.9167
afterbatch 2 length= 3
(320, 240)
(240, 320, 3)
batch =  3 stage 1 completed shape= (32, 1000)
batch 3 time= 4.2009
afterbatch 3 length= 4
(320, 240)
(240, 320, 3)
batch =  4 stage 1 completed shape= (32, 1000)
batch 4 time= 4.4792
afterbatch 4 length= 5
(320, 240)
(240, 320, 3)
batch =  5 stage 1 completed shape= (32, 1000)
batch 5 time= 4.7575
afterbatch 5 length= 6
(320, 240)
(240, 320, 3)
batch =  6 stage 1 completed shape= (32, 1000)
batch 6 time= 5.0343
afterbatch 6 length= 7
(320, 240)
(240, 320, 3)
batch =  7 stage 1 completed shape= (32, 1000)
batch 7 time= 5.3122
afterbatch 7 length= 8
(320, 240)
(240, 320, 3)
batch =  8 stage 1 completed shape= (32, 1000)


In [0]:
del model_1

In [18]:
model_2.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])



NameError: ignored

In [19]:
s= time.time()
final_array = []
for j,l in enumerate(final_list):
      img_list=[]
      for i,img in enumerate(l):
        
        
        
        img = load_img(src_2+img)
        
        if i == 0:
            print (img.size)


            
        
        
        x = img_to_array(img)
        

        
        x = x/255.
        

        if i  == 0 :
          
          print (x.shape)
          #x = cv2.cvtColor(y, cv2.COLOR_BGR2RGB)
          #x = x/127.5 -1.
        img_list.append(x)
      #print ('img_list=', len(img_list))
      testing_data = np.array(img_list)

      #print ('testing_data_shape', testing_data.shape)
      #train_data =  train_data[:,:,:,np.newaxis]
      #print (testing_data.shape)
      pred_temp = first_prediction(model_2,testing_data)

      print('batch = ', j, 'stage 1 completed', 'shape=', pred_temp.shape)
      #np.save(src_save+'temp_'+str(j)+'.npy',pred_temp)

      #print ('bacth npy saved', j)
      e = time.time()
      print ('batch',j,'time=', round(e -s ,4))

      final_array.append(pred_temp)
      print ('afterbatch',j,'length=', len(final_array))

final_array = np.concatenate(final_array)
print ('final array', final_array.shape)
np.save(src_save+'train_data_img_preprocess_MobileNetV2_2_aug.npy',final_array)
print ('totally done, train_data_img_preprocess_MobileNetV2_2_aug.npy')

(320, 240)
(240, 320, 3)
batch =  0 stage 1 completed shape= (32, 8, 10, 1280)
batch 0 time= 15.2485
afterbatch 0 length= 1
(320, 240)
(240, 320, 3)
batch =  1 stage 1 completed shape= (32, 8, 10, 1280)
batch 1 time= 26.4656
afterbatch 1 length= 2
(320, 240)
(240, 320, 3)
batch =  2 stage 1 completed shape= (32, 8, 10, 1280)
batch 2 time= 39.2956
afterbatch 2 length= 3
(320, 240)
(240, 320, 3)
batch =  3 stage 1 completed shape= (32, 8, 10, 1280)
batch 3 time= 54.1985
afterbatch 3 length= 4
(320, 240)
(240, 320, 3)
batch =  4 stage 1 completed shape= (32, 8, 10, 1280)
batch 4 time= 68.4312
afterbatch 4 length= 5
(320, 240)
(240, 320, 3)
batch =  5 stage 1 completed shape= (32, 8, 10, 1280)
batch 5 time= 81.6882
afterbatch 5 length= 6
(320, 240)
(240, 320, 3)
batch =  6 stage 1 completed shape= (32, 8, 10, 1280)
batch 6 time= 95.3798
afterbatch 6 length= 7
(320, 240)
(240, 320, 3)
batch =  7 stage 1 completed shape= (32, 8, 10, 1280)
batch 7 time= 108.6411
afterbatch 7 length= 8
(320, 2

batch =  64 stage 1 completed shape= (32, 8, 10, 1280)
batch 64 time= 818.6126
afterbatch 64 length= 65
(320, 240)
(240, 320, 3)
batch =  65 stage 1 completed shape= (32, 8, 10, 1280)
batch 65 time= 832.7231
afterbatch 65 length= 66
(320, 240)
(240, 320, 3)
batch =  66 stage 1 completed shape= (32, 8, 10, 1280)
batch 66 time= 846.7431
afterbatch 66 length= 67
(320, 240)
(240, 320, 3)
batch =  67 stage 1 completed shape= (32, 8, 10, 1280)
batch 67 time= 860.2299
afterbatch 67 length= 68
(320, 240)
(240, 320, 3)
batch =  68 stage 1 completed shape= (32, 8, 10, 1280)
batch 68 time= 873.979
afterbatch 68 length= 69
(320, 240)
(240, 320, 3)
batch =  69 stage 1 completed shape= (32, 8, 10, 1280)
batch 69 time= 888.0989
afterbatch 69 length= 70
(320, 240)
(240, 320, 3)
batch =  70 stage 1 completed shape= (32, 8, 10, 1280)
batch 70 time= 901.6806
afterbatch 70 length= 71
(320, 240)
(240, 320, 3)
batch =  71 stage 1 completed shape= (32, 8, 10, 1280)
batch 71 time= 915.2151
afterbatch 71 lengt

batch =  127 stage 1 completed shape= (32, 8, 10, 1280)
batch 127 time= 1668.7924
afterbatch 127 length= 128
(320, 240)
(240, 320, 3)
batch =  128 stage 1 completed shape= (32, 8, 10, 1280)
batch 128 time= 1683.4925
afterbatch 128 length= 129
(320, 240)
(240, 320, 3)
batch =  129 stage 1 completed shape= (32, 8, 10, 1280)
batch 129 time= 1697.0316
afterbatch 129 length= 130
(320, 240)
(240, 320, 3)
batch =  130 stage 1 completed shape= (32, 8, 10, 1280)
batch 130 time= 1710.4999
afterbatch 130 length= 131
(320, 240)
(240, 320, 3)
batch =  131 stage 1 completed shape= (32, 8, 10, 1280)
batch 131 time= 1723.7522
afterbatch 131 length= 132
(320, 240)
(240, 320, 3)
batch =  132 stage 1 completed shape= (32, 8, 10, 1280)
batch 132 time= 1738.0235
afterbatch 132 length= 133
(320, 240)
(240, 320, 3)
batch =  133 stage 1 completed shape= (32, 8, 10, 1280)
batch 133 time= 1751.7452
afterbatch 133 length= 134
(320, 240)
(240, 320, 3)
batch =  134 stage 1 completed shape= (32, 8, 10, 1280)
batch 

batch =  189 stage 1 completed shape= (32, 8, 10, 1280)
batch 189 time= 2488.6897
afterbatch 189 length= 190
(320, 240)
(240, 320, 3)
batch =  190 stage 1 completed shape= (32, 8, 10, 1280)
batch 190 time= 2503.2689
afterbatch 190 length= 191
(320, 240)
(240, 320, 3)
batch =  191 stage 1 completed shape= (32, 8, 10, 1280)
batch 191 time= 2513.8935
afterbatch 191 length= 192
(320, 240)
(240, 320, 3)
batch =  192 stage 1 completed shape= (32, 8, 10, 1280)
batch 192 time= 2524.4509
afterbatch 192 length= 193
(320, 240)
(240, 320, 3)
batch =  193 stage 1 completed shape= (32, 8, 10, 1280)
batch 193 time= 2534.7391
afterbatch 193 length= 194
(320, 240)
(240, 320, 3)
batch =  194 stage 1 completed shape= (32, 8, 10, 1280)
batch 194 time= 2545.0777
afterbatch 194 length= 195
(320, 240)
(240, 320, 3)
batch =  195 stage 1 completed shape= (32, 8, 10, 1280)
batch 195 time= 2555.6375
afterbatch 195 length= 196
(320, 240)
(240, 320, 3)
batch =  196 stage 1 completed shape= (32, 8, 10, 1280)
batch 

batch =  251 stage 1 completed shape= (32, 8, 10, 1280)
batch 251 time= 3258.7883
afterbatch 251 length= 252
(320, 240)
(240, 320, 3)
batch =  252 stage 1 completed shape= (32, 8, 10, 1280)
batch 252 time= 3269.8836
afterbatch 252 length= 253
(320, 240)
(240, 320, 3)
batch =  253 stage 1 completed shape= (32, 8, 10, 1280)
batch 253 time= 3281.8323
afterbatch 253 length= 254
(320, 240)
(240, 320, 3)
batch =  254 stage 1 completed shape= (32, 8, 10, 1280)
batch 254 time= 3294.0043
afterbatch 254 length= 255
(320, 240)
(240, 320, 3)
batch =  255 stage 1 completed shape= (32, 8, 10, 1280)
batch 255 time= 3304.8216
afterbatch 255 length= 256
(320, 240)
(240, 320, 3)
batch =  256 stage 1 completed shape= (32, 8, 10, 1280)
batch 256 time= 3316.0223
afterbatch 256 length= 257
(320, 240)
(240, 320, 3)
batch =  257 stage 1 completed shape= (32, 8, 10, 1280)
batch 257 time= 3326.8745
afterbatch 257 length= 258
(320, 240)
(240, 320, 3)
batch =  258 stage 1 completed shape= (32, 8, 10, 1280)
batch 

batch =  313 stage 1 completed shape= (32, 8, 10, 1280)
batch 313 time= 4057.1831
afterbatch 313 length= 314
(320, 240)
(240, 320, 3)
batch =  314 stage 1 completed shape= (32, 8, 10, 1280)
batch 314 time= 4069.3016
afterbatch 314 length= 315
(320, 240)
(240, 320, 3)
batch =  315 stage 1 completed shape= (32, 8, 10, 1280)
batch 315 time= 4081.7411
afterbatch 315 length= 316
(320, 240)
(240, 320, 3)
batch =  316 stage 1 completed shape= (32, 8, 10, 1280)
batch 316 time= 4094.4707
afterbatch 316 length= 317
(320, 240)
(240, 320, 3)
batch =  317 stage 1 completed shape= (32, 8, 10, 1280)
batch 317 time= 4108.0811
afterbatch 317 length= 318
(320, 240)
(240, 320, 3)
batch =  318 stage 1 completed shape= (32, 8, 10, 1280)
batch 318 time= 4120.9262
afterbatch 318 length= 319
(320, 240)
(240, 320, 3)
batch =  319 stage 1 completed shape= (32, 8, 10, 1280)
batch 319 time= 4132.397
afterbatch 319 length= 320
(320, 240)
(240, 320, 3)
batch =  320 stage 1 completed shape= (32, 8, 10, 1280)
batch 3

batch =  375 stage 1 completed shape= (32, 8, 10, 1280)
batch 375 time= 4890.1156
afterbatch 375 length= 376
(320, 240)
(240, 320, 3)
batch =  376 stage 1 completed shape= (32, 8, 10, 1280)
batch 376 time= 4902.2816
afterbatch 376 length= 377
(320, 240)
(240, 320, 3)
batch =  377 stage 1 completed shape= (32, 8, 10, 1280)
batch 377 time= 4915.7041
afterbatch 377 length= 378
(320, 240)
(240, 320, 3)
batch =  378 stage 1 completed shape= (32, 8, 10, 1280)
batch 378 time= 4926.3792
afterbatch 378 length= 379
(320, 240)
(240, 320, 3)
batch =  379 stage 1 completed shape= (32, 8, 10, 1280)
batch 379 time= 4936.9775
afterbatch 379 length= 380
(320, 240)
(240, 320, 3)
batch =  380 stage 1 completed shape= (32, 8, 10, 1280)
batch 380 time= 4948.0392
afterbatch 380 length= 381
(320, 240)
(240, 320, 3)
batch =  381 stage 1 completed shape= (32, 8, 10, 1280)
batch 381 time= 4962.0842
afterbatch 381 length= 382
(320, 240)
(240, 320, 3)
batch =  382 stage 1 completed shape= (32, 8, 10, 1280)
batch 

batch =  437 stage 1 completed shape= (32, 8, 10, 1280)
batch 437 time= 5653.7645
afterbatch 437 length= 438
(320, 240)
(240, 320, 3)
batch =  438 stage 1 completed shape= (32, 8, 10, 1280)
batch 438 time= 5667.2789
afterbatch 438 length= 439
(320, 240)
(240, 320, 3)
batch =  439 stage 1 completed shape= (32, 8, 10, 1280)
batch 439 time= 5680.9436
afterbatch 439 length= 440
(320, 240)
(240, 320, 3)
batch =  440 stage 1 completed shape= (32, 8, 10, 1280)
batch 440 time= 5693.4991
afterbatch 440 length= 441
(320, 240)
(240, 320, 3)
batch =  441 stage 1 completed shape= (32, 8, 10, 1280)
batch 441 time= 5705.5448
afterbatch 441 length= 442
(320, 240)
(240, 320, 3)
batch =  442 stage 1 completed shape= (32, 8, 10, 1280)
batch 442 time= 5716.3949
afterbatch 442 length= 443
(320, 240)
(240, 320, 3)
batch =  443 stage 1 completed shape= (32, 8, 10, 1280)
batch 443 time= 5731.1696
afterbatch 443 length= 444
(320, 240)
(240, 320, 3)
batch =  444 stage 1 completed shape= (32, 8, 10, 1280)
batch 

batch =  499 stage 1 completed shape= (32, 8, 10, 1280)
batch 499 time= 6459.5751
afterbatch 499 length= 500
(320, 240)
(240, 320, 3)
batch =  500 stage 1 completed shape= (32, 8, 10, 1280)
batch 500 time= 6471.5945
afterbatch 500 length= 501
(320, 240)
(240, 320, 3)
batch =  501 stage 1 completed shape= (32, 8, 10, 1280)
batch 501 time= 6484.3901
afterbatch 501 length= 502
(320, 240)
(240, 320, 3)
batch =  502 stage 1 completed shape= (32, 8, 10, 1280)
batch 502 time= 6496.7606
afterbatch 502 length= 503
(320, 240)
(240, 320, 3)
batch =  503 stage 1 completed shape= (32, 8, 10, 1280)
batch 503 time= 6509.0351
afterbatch 503 length= 504
(320, 240)
(240, 320, 3)
batch =  504 stage 1 completed shape= (32, 8, 10, 1280)
batch 504 time= 6520.4189
afterbatch 504 length= 505
(320, 240)
(240, 320, 3)
batch =  505 stage 1 completed shape= (32, 8, 10, 1280)
batch 505 time= 6531.9928
afterbatch 505 length= 506
(320, 240)
(240, 320, 3)
batch =  506 stage 1 completed shape= (32, 8, 10, 1280)
batch 

failure_to_label.pkl   manu_big_data		Untitled0.ipynb
label_to_category.pkl  train_data_densenet.npy	xgb_densenet12.pkl


In [0]:
label_data_0

array([ True, False, False, ..., False,  True,  True])

In [0]:
class_names= np.array(['non_defect','defect'])

In [0]:
class_names

array(['non_defect', 'defect'], dtype='<U10')

In [0]:
!pip install --upgrade keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.2.2)


Using TensorFlow backend.


'2.2.2'

In [0]:
from keras.applications.mobilenetv2 import MobileNetV2

In [0]:
from PIL import Image

In [0]:
img = Image.open(src_2+file_list[0])
img = img.resize((1920//4, 2560//4))

NameError: ignored